In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob
from PIL import Image

In [24]:
# Path
path3= glob.glob("C:/Users/abdel/medical/full dataset/*.jpg")

In [22]:
count = 0
for image in path3:
    input_img = np.array(Image.open(image))
    if(len(input_img.shape) > 2):
        # Gray Scale
        gray_scale = cv2.cvtColor(input_img,cv2.COLOR_BGR2GRAY)
        # median Filter
        median = cv2.medianBlur(gray_scale, 7)
        # Binary Threshold
        ThValue, Threshold = cv2.threshold(median, 157, 255, cv2.THRESH_BINARY)
        # Kernel
        SE = np.ones((3,3))
        # Morphological Operation
        opening_img = cv2.morphologyEx(Threshold , cv2.MORPH_OPEN ,SE )

        closing_img = cv2.morphologyEx(opening_img , cv2.MORPH_CLOSE ,SE )
        # Connected Components 
        num_labels, labels_im, stats, centroids = cv2.connectedComponentsWithStats(closing_img,connectivity=4)

        for k in range(1,num_labels):
            pts = np.where(labels_im == k)
            size = stats[k, cv2.CC_STAT_AREA]
            if size > 500 and size < 10000:
                labels_im[pts] = labels_im[pts]

            else:
                labels_im[pts] =0

        label_hue = np.uint8(179*labels_im/np.max(labels_im))
        blank_ch = 255*np.ones_like(label_hue)
        labeled_img = cv2.merge([label_hue, blank_ch, blank_ch])
        labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2BGR)                                  
        labeled_img [label_hue==0] = 0
        
        gray1 = cv2.cvtColor(labeled_img , cv2.COLOR_BGR2GRAY)
        gray2 = gray1 * gray_scale   
        opening_img1 = cv2.morphologyEx(gray2 , cv2.MORPH_OPEN ,SE )
        closing_img1 = cv2.morphologyEx(opening_img1 , cv2.MORPH_CLOSE ,SE )
        b = closing_img1 * 5
        
        # Save Data
        cv2.imwrite('C:/Users/abdel/medical/result/median/' + str(count) + '.jpg' , median)
        cv2.imwrite('C:/Users/abdel/medical/result/threshold/' + str(count) + '.jpg' , Threshold)
        cv2.imwrite('C:/Users/abdel/medical/result/opening/' + str(count) + '.jpg' , opening_img)
        cv2.imwrite('C:/Users/abdel/medical/result/closing/' + str(count) + '.jpg' , closing_img)
        cv2.imwrite('C:/Users/abdel/medical/result/labeled image/' + str(count) + '.jpg' , labeled_img)
        
        
    if(len(labeled_img[labeled_img == 255]) > 5):
        cv2.imwrite('C:/Users/abdel/medical/result/classification/tumor/' + str(count) + '.jpg' , b)
    else:
        cv2.imwrite('C:/Users/abdel/medical/result/classification/notumor/' + str(count) + '.jpg' , b)
    count = count + 1

<ipython-input-22-b92a9f41d62c>:29: RuntimeWarning: invalid value encountered in true_divide
  label_hue = np.uint8(179*labels_im/np.max(labels_im))


In [13]:
accuracy  = (32 + 33) / (32 + 33 + 26 + 0 ) * 100
print(accuracy)

71.42857142857143
